In [1]:
import sys
import time
import random
from datetime import datetime

In [2]:
import numpy as np
import pandas as pd

In [3]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.losses import binary_crossentropy
from keras import backend as K

import tensorflow as tf

In [4]:
tf.config.run_functions_eagerly(True)

***

In [5]:
# Genuine users and tweets
gen_users = pd.read_csv('dataset/users.csv')
gen_tweets = pd.read_csv('dataset/tweets.csv')

In [6]:
# User data displayed as loaded from CSV
gen_tweets

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,...,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,594073273480130560,How Randolph Hodgson and Neals Yard Dairy gave...,"<a href=""http://www.apple.com"" rel=""nofollow"">...",887281,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,1,0,Fri May 01 09:38:00 +0000 2015,2015-05-01 11:38:00,2015-05-01 12:59:53,2015-05-01 12:59:53
1,594066507723833345,“Twitter’s multi-billion dollar mistake happen...,"<a href=""http://www.apple.com"" rel=""nofollow"">...",887281,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,1,1,Fri May 01 09:11:07 +0000 2015,2015-05-01 11:11:07,2015-05-01 12:59:53,2015-05-01 12:59:53
2,593739179655323649,The evolution of advertising in the legal sect...,"<a href=""http://twitter.com/download/iphone"" r...",887281,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,1,0,Thu Apr 30 11:30:26 +0000 2015,2015-04-30 13:30:26,2015-05-01 12:59:53,2015-05-01 12:59:53
3,593737857149345792,RT @rorysutherland: Plan Bee - http://t.co/030...,"<a href=""http://twitter.com/download/iphone"" r...",887281,NaN,0,0,NaN,593693743548649472,NaN,...,NaN,NaN,NaN,0,1,4,Thu Apr 30 11:25:10 +0000 2015,2015-04-30 13:25:10,2015-05-01 12:59:53,2015-05-01 12:59:53
4,593282967134466051,RT @davewiner: Some say the Other Internet is ...,"<a href=""http://twitter.com/download/iphone"" r...",887281,NaN,0,0,NaN,593213234737446912,NaN,...,NaN,NaN,NaN,0,1,1,Wed Apr 29 05:17:36 +0000 2015,2015-04-29 07:17:36,2015-05-01 12:59:53,2015-05-01 12:59:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248528,573687545311162368,RT @FreestyIeRaps: They killed this hoe,"<a href=""http://twitter.com/download/iphone"" r...",3073742769,NaN,0,0,NaN,570307641035100161,NaN,...,NaN,NaN,NaN,0,0,1,Fri Mar 06 03:32:24 +0000 2015,2015-03-06 04:32:24,2015-05-02 01:41:44,2015-05-02 01:41:44
248529,573687532149424128,"RT @QuiNigga_: ""@SavageBars: HE KILLED IT AGAIN","<a href=""http://twitter.com/download/iphone"" r...",3073742769,NaN,0,0,NaN,572641966216884225,NaN,...,NaN,NaN,NaN,0,0,2,Fri Mar 06 03:32:20 +0000 2015,2015-03-06 04:32:20,2015-05-02 01:41:44,2015-05-02 01:41:44
248530,573687515342835712,RT @FreestyIeRaps: HE KILLED IT,"<a href=""http://twitter.com/download/iphone"" r...",3073742769,NaN,0,0,NaN,572622978447503360,NaN,...,NaN,NaN,NaN,0,0,2,Fri Mar 06 03:32:16 +0000 2015,2015-03-06 04:32:16,2015-05-02 01:41:44,2015-05-02 01:41:44
248531,573659786216079360,RT @DreecNation: S/o to my new follower @QuiNi...,"<a href=""http://twitter.com/download/iphone"" r...",3073742769,NaN,0,0,NaN,573651460103106560,NaN,...,NaN,NaN,NaN,0,0,2,Fri Mar 06 01:42:05 +0000 2015,2015-03-06 02:42:05,2015-05-02 01:41:44,2015-05-02 01:41:44


In [7]:
def create_digital_dna_from_profile(users_df):
    df = users_df
    
    # Applying necessary replacements
    # 1 - Description is available, 0 - Description not available
    # 1 - URL is available, 0 - URL is not available
    df['description'] = np.where(pd.isnull(users_df['description']) == True, 0, 1)
    df['url'] = np.where(pd.isnull(users_df['url']) == True, 0, 1)
    
    # Changed user data
    return df

def create_digital_dna_from_tweets(tweets_df):
    '''For each user id in tweets_df return a digital DNA string based on posting behaviour.'''
    
    # Add columns for counts of tweets, replies and retweets.
    tweets_df['num_retweets'] = np.where(tweets_df['retweeted_status_id'] == 0, 0, 1)
    tweets_df['num_replies'] = np.where(tweets_df['in_reply_to_status_id'] == 0, 0, 1)
    tweets_df['num_tweets'] = np.where((tweets_df['num_retweets'] == 0) & (tweets_df['num_replies'] == 0), 1, 0)

    # DNA alphabet for tweet (A), retweet (C) and reply (T).
    tweets = tweets_df['num_tweets'] == 1
    retweets = tweets_df['num_retweets'] == 1
    replies = tweets_df['num_replies'] == 1

    tweets_df.loc[:, 'DNA'] = np.where(retweets, 'C', np.where(replies, 'T', 'A'))

    # Sort tweets by timestamp.
    tweets_df = tweets_df[['user_id', 'timestamp', 'DNA']]
    tweets_df = tweets_df.sort_values(by=['timestamp'])

    # Create digital DNA string for account.
    dna = tweets_df.groupby(by=['user_id'])['DNA'].agg(lambda x: ''.join(x))
    
    return dna

In [8]:
# Assigning float valeus in fields where necessary
def time_to_number(time_str):
    return np.float32(datetime.fromisoformat(time_str).timestamp())

def number_to_float(number):
    return np.float32(number)

def time_to_float(df, column_name):
    for index, value in enumerate(df[column_name]):
        df[column_name].at[index] = time_to_number(value)

def to_float(df, column_name):
    for index, value in enumerate(df[column_name]):
        df[column_name].at[index] = number_to_float(value)

In [9]:
to_float(gen_users, "statuses_count")
to_float(gen_users, "followers_count")
to_float(gen_users, "friends_count")
to_float(gen_users, "favourites_count")
to_float(gen_users, "listed_count")

time_to_float(gen_users, "timestamp")
time_to_float(gen_users, "updated")

In [10]:
print('Users shape:', gen_users.shape)
print('Tweets shape:', gen_tweets.shape)

Users shape: (100, 42)
Tweets shape: (248533, 25)


***

In [11]:
# Filtering data which we need
filtered_user_data = gen_users.filter(["id", "statuses_count", "followers_count",
                                      "friends_count", "favourites_count", "listed_count",
                                      "url", "description", "timestamp", "updated"])

In [12]:
# Processing user data and tweets of users
processed_user_data = create_digital_dna_from_profile(filtered_user_data)
processed_user_data['tweets_dna'] = ""
processed_tweets = create_digital_dna_from_tweets(gen_tweets)

print('(Processed) Users shape:', processed_user_data.shape)
print('(Processed) Tweets shape:', processed_tweets.shape)

# Compile user data with dna tweets
for i in range(processed_user_data.shape[0]):
    user_id = processed_user_data['id'][i]
    dna = processed_tweets[user_id]
    processed_user_data['tweets_dna'].at[i] = dna

(Processed) Users shape: (100, 11)
(Processed) Tweets shape: (100,)


In [13]:
twitter_account = processed_user_data.filter(["statuses_count", "followers_count", "friends_count", 
                                              "favourites_count", "listed_count", "url", "description",
                                              "timestamp", "updated", "tweets_dna"])

## Processed Data

In [14]:
twitter_account

,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,description,timestamp,updated,tweets_dna
0,60463,568,387,15599,1,1,0,1328857216.0,1458039808.0,CCCCCCCACCCCCCAAAAAACACAAAACCCCCCCCCCACCCACCCC...
1,135,208,263,43,2,0,1,1253338112.0,1458040064.0,ACCAACACCAAAAAAAAAATTACACAAAATCTAAATAAAAAAAAAA...
2,283536,7785,424,1157,217,1,1,1241342208.0,1458039296.0,CTCCCCATACCCCTACAAAAAACACCAACAAACCCAAACTACCAAA...
3,1770,179,132,1224,0,0,0,1376278528.0,1458039680.0,CAAAACAAATAACAAAAAAAAAAACCACAAAATAAACAAAAAAAAA...
4,4790,3325,1327,235,5,1,1,1306796800.0,1458039680.0,ACAAAAAAAAAAACAAAAAAAAAAAAAAAAAAACACATCCATAAAA...
...,...,...,...,...,...,...,...,...,...,...
95,8860,178,493,715,6,0,1,1256503680.0,1458039808.0,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
96,12502,478,353,28547,0,0,1,1374653440.0,1458040192.0,ATTCAAACTTATTCAAACAACAACCACCAATCCTCATATCATCCAT...
97,1189,369,337,250,0,0,1,1423388032.0,1458039808.0,ACCCAACCCACAACACACAACTCCAAATACTTAACCCAAAAAACAA...
98,103,26,189,40,1,0,0,1340321024.0,1458039808.0,TTTTTTTTTATTTTTTTTTTTTTATAATTATTTAATATAAAATTTT...


***

## Parameters

In [15]:
input_train = twitter_account.values.reshape(twitter_account.shape[0], twitter_account.shape[1])

batch_size = 10
no_epochs = 10
validation_split = 0.1
verbosity = 1

latent_dim = 1

In [16]:
input_train.shape

(100, 10)

***

## Model

In [17]:
"""
Dropping tweets data because this model will be trianed just on the user data.
Converting all the other fields to float64 then giving it the shape for input.
The final variable contains the numpy array made from the dataframe.
"""
training_data = twitter_account.drop(['tweets_dna'], axis = 1)
training_data = training_data.apply(pd.to_numeric)
training_data = training_data.values.reshape(training_data.shape[0], training_data.shape[1])
features_in_feature_vector = training_data.shape[1]
input_shape = (features_in_feature_vector, )

## Encoder

In [18]:
i       = Input(shape=input_shape, name='encoder_input')
x       = Dense(features_in_feature_vector, activation='relu')(i)
x       = BatchNormalization()(x)
mu      = Dense(latent_dim, name='latent_mu')(x)
sigma   = Dense(latent_dim, name='latent_sigma')(x)

In [19]:
# Define sampling with reparameterization trick
def sample_z(args):
  mu, sigma = args
  batch     = K.shape(mu)[0]
  dim       = K.int_shape(mu)[1]
  eps       = K.random_normal(shape=(batch, dim))
  return mu + K.exp(sigma / 2) * eps

In [20]:
# Reparameterization for correct gradient
z = Lambda(sample_z, output_shape=(latent_dim, ), name='z')([mu, sigma])

In [21]:
# Instantiate encoder
encoder = Model(i, [mu, sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 9)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 9)            90          encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 9)            36          dense[0][0]                      
__________________________________________________________________________________________________
latent_mu (Dense)               (None, 1)            10          batch_normalization[0][0]        
____________________________________________________________________________________________

## Decoder

In [22]:
d_i   = Input(shape=(latent_dim, ), name='decoder_input')
x     = Dense(features_in_feature_vector, activation='relu')(d_i)
x     = Reshape(input_shape)(x)
o     = BatchNormalization(name='decoder_output')(x)

In [23]:
# Instantiate decoder
decoder = Model(d_i, o, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 1)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 18        
_________________________________________________________________
reshape (Reshape)            (None, 9)                 0         
_________________________________________________________________
decoder_output (BatchNormali (None, 9)                 36        
Total params: 54
Trainable params: 36
Non-trainable params: 18
_________________________________________________________________


# VAE

In [24]:
# Instantiate VAE
vae_outputs = decoder(encoder(i)[2])
vae         = Model(i, vae_outputs, name='vae')
vae.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 9)]               0         
_________________________________________________________________
encoder (Functional)         [(None, 1), (None, 1), (N 146       
_________________________________________________________________
decoder (Functional)         (None, 9)                 54        
Total params: 200
Trainable params: 164
Non-trainable params: 36
_________________________________________________________________


## Loss Function

In [25]:
"""
We are calcuating the binary crossentropy here.
KL Divergence and Reconstruction are the two
metrics which we are using to make prediction.
"""
def kl_reconstruction_loss(true, pred):
  # Reconstruction loss
  reconstruction_loss = binary_crossentropy(K.flatten(true), K.flatten(pred)) * input_shape[0]
  # KL divergence loss
  kl_loss = 1 + sigma - K.square(mu) - K.exp(sigma)
  kl_loss = K.sum(kl_loss, axis=-1)
  kl_loss *= -0.5
  # Total loss = 50% rec + 50% KL divergence loss
  return K.mean(reconstruction_loss + kl_loss)

## Training

In [26]:
"""
We are using the adam optimizer from Keras in combination
of our own defined loss function which is the mean of
reconstruction loss and the KL divergence.
"""
vae.compile(optimizer='adam', loss=kl_reconstruction_loss)

In [27]:
vae.fit(x = training_data, y = training_data, 
        epochs = no_epochs, 
        batch_size = batch_size, 
        validation_split = validation_split)

c:\users\bilal\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


Epoch 1/10
9/9 [==============================] - 6s 614ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 4s 437ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 4s 441ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 4s 485ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
9/9 [==============================] - 4s 459ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/10
9/9 [==============================] - 4s 446ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
9/9 [==============================] - 14s 2s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/10
9/9 [==============================] - 8s 840ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
9/9 [==============================] - 4s 435ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/10
9/9 [==============================]